## Lab 3: DDI 

### Imports

In [1]:
import os
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.corpus import stopwords 
import xml.etree.ElementTree as ET
from nltk.tree import Tree

### Variables

In [98]:
inputdir = '../../data/Devel'
test_path = '../../data/Test-DDI'
train_path = '../../data/Train'
outputfile = 'task9.2_develGoal_1.txt'

my_parser = CoreNLPDependencyParser(url="http://localhost:9000")

### Functions

In [145]:
def analyze(sent):
    if len(sent)<= 0:
        return None
    mytree, = my_parser.raw_parse(sent)
    tree = mytree.nodes
    ini_token = 0
                   
    # clean tree
    aux = ['address', 'head', 'lemma', 'rel', 'word', 'tag']
    for k in range(1,len(tree)):
        node = tree[k] 
        for key in list(node):
            if key not in aux:
                del node[key]
        
        # add offset
        ini_token = sent.find(node['word'] ,ini_token)
        
        node['start'] = ini_token
        ini_token += len(node['word'])
        node['end'] = ini_token
        print(node)

    return tree

def check_interaction(analysis, entities,e1,e2):
    interaction = -1
    # Rules (with)
    
    return interaction

def evaluate(inputdir, outputfile):
    os.system("java -jar eval/evaluateDDI.jar "+ str(inputdir) + " " + str(outputfile))
    return

def main_function(inputdir):
    # process each file in directory
    for filename in os.listdir(inputdir):
        # parse XML file, obtaining a DOM tree
        fullname = os.path.join(inputdir, filename)
        tree = ET.parse(fullname)
        root = tree.getroot()  
        
        # process each sentence in the file
        for sentence in root.findall('sentence'):
            # Get sentence id and tokenize text
            sent_id = sentence.get('id') # get sentence id
            sent_text = sentence.get('text') #get sentence text 
            # load sentence entities into a dictionary
            entities = {}
            for ent in root.findall('entity'):
                ent_id = ent.get('id') 
                offs = ent.get('charOffset').value.split('-')
                entities[ent_id] = offs
            # Tokenize, tag and parse sentence
            analysis = analyze(sent_text)
            
            # for each pair in the sentence, decide whether it is DDI and its type
            for pair in root.findall('pair'):
                id_e1 = pair.get('e1').value
                id_e2 = pair.get('e2').value
                (is_ddi, ddi_type) = check_interaction(analysis, entities, id_e1, id_e2)
    # get performance score
    evaluate(inputdir, outputfile)

In [146]:
main_function(inputdir)

{'address': 1, 'word': 'The', 'lemma': 'the', 'tag': 'DT', 'head': 2, 'rel': 'det', 'start': 0, 'end': 3}
{'address': 2, 'word': 'fluoroquinolones', 'lemma': 'fluoroquinolone', 'tag': 'NNS', 'head': 0, 'rel': 'ROOT', 'start': 4, 'end': 20}
{'address': 3, 'word': 'for', 'lemma': 'for', 'tag': 'IN', 'head': 6, 'rel': 'case', 'start': 21, 'end': 24}
{'address': 4, 'word': 'urinary', 'lemma': 'urinary', 'tag': 'JJ', 'head': 6, 'rel': 'amod', 'start': 25, 'end': 32}
{'address': 5, 'word': 'tract', 'lemma': 'tract', 'tag': 'NN', 'head': 6, 'rel': 'compound', 'start': 33, 'end': 38}
{'address': 6, 'word': 'infections', 'lemma': 'infection', 'tag': 'NNS', 'head': 2, 'rel': 'nmod', 'start': 39, 'end': 49}
{'address': 7, 'word': ':', 'lemma': ':', 'tag': ':', 'head': 2, 'rel': 'punct', 'start': 49, 'end': 50}
{'address': 8, 'word': 'a', 'lemma': 'a', 'tag': 'DT', 'head': 9, 'rel': 'det', 'start': 51, 'end': 52}
{'address': 9, 'word': 'review', 'lemma': 'review', 'tag': 'NN', 'head': 2, 'rel': 'd

{'address': 1, 'word': 'After', 'lemma': 'after', 'tag': 'IN', 'head': 2, 'rel': 'case', 'start': 0, 'end': 5}
{'address': 2, 'word': 'incubation', 'lemma': 'incubation', 'tag': 'NN', 'head': 12, 'rel': 'nmod', 'start': 6, 'end': 16}
{'address': 3, 'word': 'at', 'lemma': 'at', 'tag': 'IN', 'head': 6, 'rel': 'case', 'start': 17, 'end': 19}
{'address': 4, 'word': '0', 'lemma': '0', 'tag': 'CD', 'head': 6, 'rel': 'nummod', 'start': 20, 'end': 21}
{'address': 5, 'word': 'degrees', 'lemma': 'degree', 'tag': 'NNS', 'head': 6, 'rel': 'compound', 'start': 22, 'end': 29}
{'address': 6, 'word': 'C', 'lemma': 'c', 'tag': 'NN', 'head': 2, 'rel': 'nmod', 'start': 30, 'end': 31}
{'address': 7, 'word': 'for', 'lemma': 'for', 'tag': 'IN', 'head': 9, 'rel': 'case', 'start': 32, 'end': 35}
{'address': 8, 'word': '4', 'lemma': '4', 'tag': 'CD', 'head': 9, 'rel': 'nummod', 'start': 36, 'end': 37}
{'address': 9, 'word': 'h', 'lemma': 'h', 'tag': 'NN', 'head': 2, 'rel': 'nmod', 'start': 38, 'end': 39}
{'add

{'address': 1, 'word': '5', 'lemma': '5', 'tag': 'CD', 'head': 0, 'rel': 'ROOT', 'start': 0, 'end': 1}
{'address': 2, 'word': '.', 'lemma': '.', 'tag': '.', 'head': 1, 'rel': 'punct', 'start': 1, 'end': 2}
{'address': 1, 'word': 'Maximal', 'lemma': 'maximal', 'tag': 'JJ', 'head': 3, 'rel': 'amod', 'start': 0, 'end': 7}
{'address': 2, 'word': 'exercise', 'lemma': 'exercise', 'tag': 'NN', 'head': 3, 'rel': 'compound', 'start': 8, 'end': 16}
{'address': 3, 'word': 'testing', 'lemma': 'testing', 'tag': 'NN', 'head': 16, 'rel': 'nsubj', 'start': 17, 'end': 24}
{'address': 4, 'word': ',', 'lemma': ',', 'tag': ',', 'head': 3, 'rel': 'punct', 'start': 24, 'end': 25}
{'address': 5, 'word': 'a', 'lemma': 'a', 'tag': 'DT', 'head': 6, 'rel': 'det', 'start': 26, 'end': 27}
{'address': 6, 'word': 'maneuver', 'lemma': 'maneuver', 'tag': 'NN', 'head': 8, 'rel': 'nsubj', 'start': 28, 'end': 36}
{'address': 7, 'word': 'often', 'lemma': 'often', 'tag': 'RB', 'head': 8, 'rel': 'advmod', 'start': 37, 'end'

{'address': 8, 'word': 'efficacy', 'lemma': 'efficacy', 'tag': 'NN', 'head': 5, 'rel': 'nmod', 'start': 49, 'end': 57}
{'address': 9, 'word': 'of', 'lemma': 'of', 'tag': 'IN', 'head': 10, 'rel': 'case', 'start': 58, 'end': 60}
{'address': 10, 'word': 'antipsychotics', 'lemma': 'antipsychotic', 'tag': 'NNS', 'head': 8, 'rel': 'nmod', 'start': 61, 'end': 75}
{'address': 11, 'word': ',', 'lemma': ',', 'tag': ',', 'head': 10, 'rel': 'punct', 'start': 75, 'end': 76}
{'address': 12, 'word': 'future', 'lemma': 'future', 'tag': 'JJ', 'head': 13, 'rel': 'amod', 'start': 77, 'end': 83}
{'address': 13, 'word': 'studies', 'lemma': 'study', 'tag': 'NNS', 'head': 15, 'rel': 'nsubj', 'start': 84, 'end': 91}
{'address': 14, 'word': 'should', 'lemma': 'should', 'tag': 'MD', 'head': 15, 'rel': 'aux', 'start': 92, 'end': 98}
{'address': 15, 'word': 'focus', 'lemma': 'focus', 'tag': 'VB', 'head': 10, 'rel': 'acl:relcl', 'start': 99, 'end': 104}
{'address': 16, 'word': 'on', 'lemma': 'on', 'tag': 'IN', 'he

{'address': 1, 'word': 'These', 'lemma': 'these', 'tag': 'DT', 'head': 2, 'rel': 'det', 'start': 0, 'end': 5}
{'address': 2, 'word': 'results', 'lemma': 'result', 'tag': 'NNS', 'head': 3, 'rel': 'nsubj', 'start': 6, 'end': 13}
{'address': 3, 'word': 'suggest', 'lemma': 'suggest', 'tag': 'VBP', 'head': 0, 'rel': 'ROOT', 'start': 14, 'end': 21}
{'address': 4, 'word': 'that', 'lemma': 'that', 'tag': 'IN', 'head': 6, 'rel': 'mark', 'start': 22, 'end': 26}
{'address': 5, 'word': 'KRM-1648', 'lemma': 'krm-1648', 'tag': 'NN', 'head': 6, 'rel': 'nsubj', 'start': 27, 'end': 35}
{'address': 6, 'word': 'has', 'lemma': 'have', 'tag': 'VBZ', 'head': 3, 'rel': 'ccomp', 'start': 36, 'end': 39}
{'address': 7, 'word': 'a', 'lemma': 'a', 'tag': 'DT', 'head': 9, 'rel': 'det', 'start': 40, 'end': 41}
{'address': 8, 'word': 'great', 'lemma': 'great', 'tag': 'JJ', 'head': 9, 'rel': 'amod', 'start': 42, 'end': 47}
{'address': 9, 'word': 'potential', 'lemma': 'potential', 'tag': 'JJ', 'head': 6, 'rel': 'dobj'

...

{'address': 1, 'word': 'Caution', 'lemma': 'caution', 'tag': 'NN', 'head': 4, 'rel': 'nsubjpass', 'start': 0, 'end': 7}
{'address': 2, 'word': 'should', 'lemma': 'should', 'tag': 'MD', 'head': 4, 'rel': 'aux', 'start': 8, 'end': 14}
{'address': 3, 'word': 'be', 'lemma': 'be', 'tag': 'VB', 'head': 4, 'rel': 'auxpass', 'start': 15, 'end': 17}
{'address': 4, 'word': 'used', 'lemma': 'use', 'tag': 'VBN', 'head': 0, 'rel': 'ROOT', 'start': 18, 'end': 22}
{'address': 5, 'word': 'if', 'lemma': 'if', 'tag': 'IN', 'head': 8, 'rel': 'mark', 'start': 23, 'end': 25}
{'address': 6, 'word': 'naproxen', 'lemma': 'naproxen', 'tag': 'NN', 'head': 8, 'rel': 'nsubjpass', 'start': 26, 'end': 34}
{'address': 7, 'word': 'is', 'lemma': 'be', 'tag': 'VBZ', 'head': 8, 'rel': 'auxpass', 'start': 35, 'end': 37}
{'address': 8, 'word': 'administered', 'lemma': 'administer', 'tag': 'VBN', 'head': 4, 'rel': 'advcl', 'start': 38, 'end': 50}
{'address': 9, 'word': 'concomitantly', 'lemma': 'concomitantly', 'tag': 'RB'

{'address': 1, 'word': 'Coadministration', 'lemma': 'coadministration', 'tag': 'NN', 'head': 19, 'rel': 'nsubj', 'start': 0, 'end': 16}
{'address': 2, 'word': 'of', 'lemma': 'of', 'tag': 'IN', 'head': 3, 'rel': 'case', 'start': 17, 'end': 19}
{'address': 3, 'word': 'VIRACEPT', 'lemma': 'VIRACEPT', 'tag': 'NNP', 'head': 1, 'rel': 'nmod', 'start': 20, 'end': 28}
{'address': 4, 'word': 'and', 'lemma': 'and', 'tag': 'CC', 'head': 3, 'rel': 'cc', 'start': 29, 'end': 32}
{'address': 5, 'word': 'drugs', 'lemma': 'drug', 'tag': 'NNS', 'head': 3, 'rel': 'conj', 'start': 33, 'end': 38}
{'address': 6, 'word': 'primarily', 'lemma': 'primarily', 'tag': 'RB', 'head': 7, 'rel': 'advmod', 'start': 39, 'end': 48}
{'address': 7, 'word': 'metabolized', 'lemma': 'metabolize', 'tag': 'VBN', 'head': 1, 'rel': 'acl', 'start': 49, 'end': 60}
{'address': 8, 'word': 'by', 'lemma': 'by', 'tag': 'IN', 'head': 9, 'rel': 'case', 'start': 61, 'end': 63}
{'address': 9, 'word': 'CYP3A', 'lemma': 'cyp3a', 'tag': 'NN', 

{'address': 1, 'word': 'therefore', 'lemma': 'therefore', 'tag': 'RB', 'head': 7, 'rel': 'advmod', 'start': 0, 'end': 9}
{'address': 2, 'word': ',', 'lemma': ',', 'tag': ',', 'head': 7, 'rel': 'punct', 'start': 9, 'end': 10}
{'address': 3, 'word': 'VIRACEPT', 'lemma': 'VIRACEPT', 'tag': 'NNP', 'head': 7, 'rel': 'nsubjpass', 'start': 11, 'end': 19}
{'address': 4, 'word': 'should', 'lemma': 'should', 'tag': 'MD', 'head': 7, 'rel': 'aux', 'start': 20, 'end': 26}
{'address': 5, 'word': 'not', 'lemma': 'not', 'tag': 'RB', 'head': 7, 'rel': 'neg', 'start': 27, 'end': 30}
{'address': 6, 'word': 'be', 'lemma': 'be', 'tag': 'VB', 'head': 7, 'rel': 'auxpass', 'start': 31, 'end': 33}
{'address': 7, 'word': 'administered', 'lemma': 'administer', 'tag': 'VBN', 'head': 0, 'rel': 'ROOT', 'start': 34, 'end': 46}
{'address': 8, 'word': 'concurrently', 'lemma': 'concurrently', 'tag': 'RB', 'head': 7, 'rel': 'advmod', 'start': 47, 'end': 59}
{'address': 9, 'word': 'with', 'lemma': 'with', 'tag': 'IN', 'h

{'address': 1, 'word': 'Currently', 'lemma': 'currently', 'tag': 'RB', 'head': 4, 'rel': 'advmod', 'start': 0, 'end': 9}
{'address': 2, 'word': ',', 'lemma': ',', 'tag': ',', 'head': 4, 'rel': 'punct', 'start': 9, 'end': 10}
{'address': 3, 'word': 'there', 'lemma': 'there', 'tag': 'EX', 'head': 4, 'rel': 'expl', 'start': 11, 'end': 16}
{'address': 4, 'word': 'are', 'lemma': 'be', 'tag': 'VBP', 'head': 0, 'rel': 'ROOT', 'start': 17, 'end': 20}
{'address': 5, 'word': 'no', 'lemma': 'no', 'tag': 'DT', 'head': 6, 'rel': 'neg', 'start': 21, 'end': 23}
{'address': 6, 'word': 'safety', 'lemma': 'safety', 'tag': 'NN', 'head': 4, 'rel': 'nsubj', 'start': 24, 'end': 30}
{'address': 7, 'word': 'and', 'lemma': 'and', 'tag': 'CC', 'head': 6, 'rel': 'cc', 'start': 31, 'end': 34}
{'address': 8, 'word': 'efficacy', 'lemma': 'efficacy', 'tag': 'NN', 'head': 9, 'rel': 'compound', 'start': 35, 'end': 43}
{'address': 9, 'word': 'data', 'lemma': 'datum', 'tag': 'NNS', 'head': 6, 'rel': 'conj', 'start': 44,

{'address': 3, 'word': 'recommended', 'lemma': 'recommend', 'tag': 'VBN', 'head': 0, 'rel': 'ROOT', 'start': 6, 'end': 17}
{'address': 4, 'word': 'that', 'lemma': 'that', 'tag': 'IN', 'head': 10, 'rel': 'mark', 'start': 18, 'end': 22}
{'address': 5, 'word': 'the', 'lemma': 'the', 'tag': 'DT', 'head': 6, 'rel': 'det', 'start': 23, 'end': 26}
{'address': 6, 'word': 'dose', 'lemma': 'dose', 'tag': 'NN', 'head': 10, 'rel': 'nsubjpass', 'start': 27, 'end': 31}
{'address': 7, 'word': 'of', 'lemma': 'of', 'tag': 'IN', 'head': 8, 'rel': 'case', 'start': 32, 'end': 34}
{'address': 8, 'word': 'rifabutin', 'lemma': 'rifabutin', 'tag': 'NN', 'head': 6, 'rel': 'nmod', 'start': 35, 'end': 44}
{'address': 9, 'word': 'be', 'lemma': 'be', 'tag': 'VB', 'head': 10, 'rel': 'auxpass', 'start': 45, 'end': 47}
{'address': 10, 'word': 'reduced', 'lemma': 'reduce', 'tag': 'VBN', 'head': 3, 'rel': 'ccomp', 'start': 48, 'end': 55}
{'address': 11, 'word': 'to', 'lemma': 'to', 'tag': 'TO', 'head': 12, 'rel': 'case